# Geocoding missing schools


In [79]:
# my usual imports and setups
import pylab as pl
import pandas as pd
import numpy as np
#imports downloader

import scipy.stats
import geopy 
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

import time


%pylab inline

import os

Populating the interactive namespace from numpy and matplotlib


# Read in data



In [141]:
# REad in Data
ref = 'MissingSchoolsForRevGeo.csv'
df = pd.read_csv(ref)

# Format  and clean Dataset
#df.rename(columns = {'Unnamed: 0':"MasterIndex"}, inplace = True)
df.drop(['Unnamed: 0',], axis=1, inplace=True)

# Define Unique Schools List
UniqueSchools = df.drop_duplicates([u'SCHNAM'])#.reset_index()
UniqueSchools = UniqueSchools[~UniqueSchools.SCHNAM.isnull()]

# Reset Lat Long
UniqueSchools['Latitude'] = np.nan
UniqueSchools['Longitude'] = np.nan

# Add Burrah Column
burrah = {'M':'Manhattan','X':'Bronx','K':'Brooklyn','Q':'Queens','R':'Staten Island'}
UniqueSchools['Borough'] = UniqueSchools.DBN.apply(lambda x : burrah[x[2:3]])

In [142]:
UniqueSchools.head()

,index,MasterIndex,DBN,SCHNAM,Latitude,Longitude,Borough
0,0,0,00M801,HUNTER SCHOOL,NaN,NaN,Manhattan
1,3,6,01M022,GUSTAVE STRAUBENMULLER,NaN,NaN,Manhattan
2,4,7,01M025,MARTA VALLE,NaN,NaN,Manhattan
3,5,9,01M056,CORLEARS - JHS #56,NaN,NaN,Manhattan
4,6,10,01M060,OTTILIA M. BEHA,NaN,NaN,Manhattan


In [143]:
UniqueSchools.tail()

,index,MasterIndex,DBN,SCHNAM,Latitude,Longitude,Borough
601,690,3474,29Q381,DISTRICT 29 PRE-K CENTER,NaN,NaN,Queens
602,691,3475,30Q361,THE WOODSIDE COMMUNITY SCHOOL,NaN,NaN,Queens
603,692,3476,30Q389,DISTRICT 30 PRE-K CENTER,NaN,NaN,Queens
604,693,3477,31R062,THE KATHLEEN GRIMM SCHOOL FOR LEADERSHIP AND S...,NaN,NaN,Staten Island
605,694,3478,31R066,THE RICHMOND PRE-K CENTER,NaN,NaN,Staten Island


In [144]:
UniqueSchools.columns

Index(['index', 'MasterIndex', 'DBN', 'SCHNAM', 'Latitude', 'Longitude',
       'Borough'],
      dtype='object')

In [145]:
UniqueSchools.shape

(605, 7)

Let's find the school coordinates using a reverse geocode technique.

In [146]:
geolocator = Nominatim()



In [147]:
def do_geocode(address, count):
    
    try:
        return geolocator.geocode(address)
    except GeocoderTimedOut:
        count += 1
        return do_geocode(address)
    
    

In [148]:
UniqueSchools.columns

Index(['index', 'MasterIndex', 'DBN', 'SCHNAM', 'Latitude', 'Longitude',
       'Borough'],
      dtype='object')

In [271]:

for i in UniqueSchools.index:
    
    location = geolocator.geocode(UniqueSchools[u'SCHNAM'][i].lower() + ' ' + 
                                  UniqueSchools[u'Borough'][i] + ' NY')
#     location = do_geocode(UniqueSchools[u'SCHNAM'][i] + 
#                           ' New York City')   
        
   
    if location is not None:
        UniqueSchools['Latitude'][i] = location.latitude
        UniqueSchools['Longitude'][i] = location.longitude
    else:
        UniqueSchools['Latitude'][i] = 'Missing'
        UniqueSchools['Longitude'][i] = 'Missing'
        
    # Sleep to ensure meet OSM requirements of no more than 1 per second
    time.sleep(1.1)


/nfshome/adn323/.conda/envs/Python3_adn323/lib/python3.5/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/nfshome/adn323/.conda/envs/Python3_adn323/lib/python3.5/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/nfshome/adn323/.conda/envs/Python3_adn323/lib/python3.5/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back 

In [272]:
UniqueSchools.head()

,index,MasterIndex,DBN,SCHNAM,Latitude,Longitude,Borough
0,0,0,00M801,HUNTER SCHOOL,Missing,Missing,Manhattan
1,3,6,01M022,GUSTAVE STRAUBENMULLER,40.7191,-73.98,Manhattan
2,4,7,01M025,MARTA VALLE,Missing,Missing,Manhattan
3,5,9,01M056,CORLEARS - JHS #56,Missing,Missing,Manhattan
4,6,10,01M060,OTTILIA M. BEHA,Missing,Missing,Manhattan


In [273]:
UniqueSchools.tail()

,index,MasterIndex,DBN,SCHNAM,Latitude,Longitude,Borough
601,690,3474,29Q381,DISTRICT 29 PRE-K CENTER,Missing,Missing,Queens
602,691,3475,30Q361,THE WOODSIDE COMMUNITY SCHOOL,Missing,Missing,Queens
603,692,3476,30Q389,DISTRICT 30 PRE-K CENTER,Missing,Missing,Queens
604,693,3477,31R062,THE KATHLEEN GRIMM SCHOOL FOR LEADERSHIP AND S...,Missing,Missing,Staten Island
605,694,3478,31R066,THE RICHMOND PRE-K CENTER,Missing,Missing,Staten Island


In [276]:
# Save Dataframe to csv
UniqueSchools.to_csv('MissingsSchoolsLoc.csv', sep=',')

In [267]:
# TEst individual data points
x=0

print(UniqueSchools['SCHNAM'][x])
location = geolocator.geocode(UniqueSchools['DBN'][x] + ' ' + UniqueSchools['SCHNAM'][x] + ' New York' )#+ UniqueSchools[u'Borough'][x])
print(location)
#print(location.latitude, location.longitude)

HUNTER SCHOOL
None
